# Workbook

This notebook contains all the necessary code (reading data, preprocessing, padding, masking, etc.) from [dataset_and_models.ipynb](dataset_and_models.ipynb) to get started on building and training the neural networks for the excercises in [labday.md](labday.md). The utility functions are now imported from python modules. It is advised to also put your model definitions in a module once finalized.

As mentioned there, make sure to:
* define your Model(s) with configurable hyperparameters as functions or classes.
* be able to save/load your model and all states necessary to evaluate this later on an independent test data set.

It also makes sense to copy this notebook (or copy the code into script/modules) for building different models (e.g. the Deep Set NN from task 2 and the GCN from task 3).

# Reading, Converting and Mapping

In [ ]:
# autoreload changed modules (see https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html for caveats)
%load_ext autoreload
%autoreload all

In [ ]:
import json
from pathlib import Path

import awkward as ak
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split

from utils import (
    load_data,
    get_adj,
    preprocess,
    GraphDataset,
    collate_fn,
    fit,
)
# TODO: does it need credentials for git clone with ssh?

In [ ]:
feature_columns = ["prodTime", "x", "y", "z", "energy", "px", "py", "pz"]

In [ ]:
df, labels = load_data("smartbkg_dataset_4k.parquet", row_groups=[0])

In [ ]:
with open("pdg_mapping.json") as f:
    pdg_mapping = dict(json.load(f))

In [ ]:
data = preprocess(df, pdg_mapping=pdg_mapping, feature_columns=feature_columns)

# Adjacency Matrix and GCN

For GCN models we also need the adjacency matrices (for others it can be ignored)

In [ ]:
data["adj"] = [get_adj(index, mother) for index, mother in zip(data["index"], data["mother"])]

# Build a model
Example models from the introductory notebook are provided in `models.py`. Adjust them to your needs

In [ ]:
from models import from_config

It's good practice to make your model setups reproducible from a single configuration which you can store to reproduce your model later, e.g:

In [ ]:
save_path = Path("saved_models")
save_path.mkdir(exist_ok=True)
tag = "deepset" # name identifying the setup (used to name the folder to save model and configuration to)
model_path = save_path / tag
model_path.mkdir(exist_ok=True)

In [ ]:
config = {
    "model_name": "deepset",
    "units": 32,
}

In [ ]:
with open(model_path / f"config.json", "w") as f:
    json.dump(config, f)

In [ ]:
model = from_config(config) # build model from config

In [ ]:
model

# Fit Model
This section features the generator for the gcn models that provides `features`, `pdg_mapped` and `adj` - you can also use it for models that don't use all inputs, if you define the `forward` function to take a batch as a dictionary, ignoring the unused keys.

In [ ]:
data_train = {}
data_val = {}
(
    data_train["features"], data_val["features"],
    data_train["pdg_mapped"], data_val["pdg_mapped"],
    data_train["adj"], data_val["adj"],
    y_train, y_val
) = train_test_split(data["features"], data["pdg_mapped"], data["adj"], labels)

In [ ]:
dl_train, dl_val = [
    torch.utils.data.DataLoader(
        GraphDataset(feat=x["features"], pdg=x["pdg_mapped"], adj=x["adj"], y=y),
        batch_size=256,
        collate_fn=collate_fn
    )
    for x, y in [(data_train, y_train), (data_val, y_val)]
]

In [ ]:
history = []

In [ ]:
history = fit(model, dl_train, dl_val, epochs=10, history=history)

In [ ]:
pd.DataFrame(history)

In [ ]:
df_history = pd.DataFrame(history)
df_history.plot()

In [ ]:
df_history.to_csv(model_path / "history.csv")

After training you should then save your model weights

In [ ]:
torch.save(model.state_dict(), model_path / "state.pt")